In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Iceberg") \
    .master("spark://spark-iceberg:7077") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hadoop") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9001") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("PySpark started")

Apenas exemplos e testes apartir de agora

In [ ]:
print(spark.version)

In [ ]:
spark.sql("SHOW TABLES IN iceberg.default").show()

In [ ]:
data = [("Alice", 30), ("Bob", 25), ("Charlie", 35)]
columns = ["name", "age"]

df = spark.createDataFrame(data, columns)

table_name = "iceberg.default.people"

df.writeTo(table_name).using("iceberg").create()

In [ ]:
spark.sql(f"SHOW TABLES IN iceberg.default").show()

In [ ]:
spark.sql(f"SELECT * FROM {table_name}").show()